<a href="https://colab.research.google.com/github/cuie23/learning-pytorch/blob/main/torch_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Outline of how to use a modular approach to model buildling

In [1]:
import os
import requests
import zipfile
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [2]:
# Setup path to data folder
data_path = Path("folder")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Did not find folder/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


In [3]:
modular = Path("modular")
if not(modular.is_dir()):
  modular.mkdir(parents=True, exist_ok=True)  # make modular directory

In [4]:
%%writefile modular/data_setup.py

import os
import torch
from torch.utils.data import DataLoader
from torchvision  import datasets, transforms

# Create PyTorch DataLoaders for image classification data.
NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):

  # Use ImageFolder to create datasets
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes  # list of target classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Writing modular/data_setup.py


Code to build model

In [5]:
%%writefile modular/model_builder.py

import torch
from torch import nn

# CNN
class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x

Writing modular/model_builder.py


Can directly import TinyVGG from *modular*

In [6]:
from modular import model_builder
device = "cuda" if torch.cuda.is_available() else "cpu"

class_names = ['1', '2', '3']

model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=10,
                              output_shape=len(class_names)).to(device)

Code for training/testing

In [7]:
%%writefile modular/engine.py
# code for training/testing

import torch
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  model.train()
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    print(f"Training batch: {batch + 1}/{len(dataloader)}")

    y_pred = model(X)

    loss = loss_fn(y_pred, y)
    train_loss += loss.item()   # accumulate loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim = 1), dim = 1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  if len(dataloader) == 0:
    return 0, 0

  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:

  model.eval()

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
      for batch, (X, y) in enumerate(dataloader):
          X, y = X.to(device), y.to(device)

          test_pred = model(X)

          loss = loss_fn(test_pred, y)
          test_loss += loss.item()

          test_pred_labels = test_pred.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
        train_dataloader: torch.utils.data.DataLoader,
        test_dataloader: torch.utils.data.DataLoader,
        optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module,
        epochs: int,
        device: torch.device) -> Dict[str, List]:

  # results dict
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  for i in range(epochs):
    train_loss, train_acc = train_step(model = model,
                                       dataloader = train_dataloader,
                                       loss_fn = loss_fn,
                                       optimizer = optimizer,
                                       device  = device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    print(
          f"Epoch: {i+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  return results

Writing modular/engine.py


Save model

In [8]:
%%writefile modular/utils.py

import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):

  # Create directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  assert model_name.endswith(".pth") or model_name.endswith(".pt")
  model_save_path = target_dir_path / model_name

  print(f"Saving model to \'{model_save_path}\'")
  torch.save(obj=model.state_dict(), f=model_save_path)

Writing modular/utils.py


Do entire create + train/test + save

In [20]:
def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(Path("modular"))

There are 1 directories and 5 images in 'modular'.
There are 0 directories and 5 images in 'modular/__pycache__'.


In [30]:
%%writefile train.py

import os
import torch
from modular import data_setup, engine, model_builder, utils
from torchvision import transforms

# hyperparams
NUM_EPOCHS = 30
BATCH_SIZE = 64
HIDDEN_UNITS = 10
LEARNING_RATE = 0.01

# Set up other input variables
train_dir = "folder/pizza_steak_sushi/train"
test_dir = "folder/pizza_steak_sushi/test"

device = "cuda" if torch.cuda.is_available() else "cpu"

data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Training
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

print("hi")

# Save model
utils.save_model(model=model,
                 target_dir="models",
                 model_name="trained_model.pth")

Overwriting train.py


In [31]:
!python train.py

Training batch: 1/4
Training batch: 2/4
Training batch: 3/4
Training batch: 4/4
Epoch: 1 | train_loss: 1.1698 | train_acc: 0.3448 | test_loss: 1.0929 | test_acc: 0.1484
Training batch: 1/4
Training batch: 2/4
Training batch: 3/4
Training batch: 4/4
Epoch: 2 | train_loss: 1.1088 | train_acc: 0.3448 | test_loss: 1.0964 | test_acc: 0.4837
Training batch: 1/4
Training batch: 2/4
Training batch: 3/4
Training batch: 4/4
Epoch: 3 | train_loss: 1.0929 | train_acc: 0.3795 | test_loss: 1.0946 | test_acc: 0.2940
Training batch: 1/4
Training batch: 2/4
Training batch: 3/4
Training batch: 4/4
Epoch: 4 | train_loss: 1.0510 | train_acc: 0.4760 | test_loss: 1.0595 | test_acc: 0.5305
Training batch: 1/4
Training batch: 2/4
Training batch: 3/4
Training batch: 4/4
Epoch: 5 | train_loss: 0.9525 | train_acc: 0.6044 | test_loss: 1.0310 | test_acc: 0.5291
Training batch: 1/4
Training batch: 2/4
Training batch: 3/4
Training batch: 4/4
Epoch: 6 | train_loss: 0.9160 | train_acc: 0.5936 | test_loss: 0.9701 | tes